<a name="top"></a>Contents
===
- [Introduction](#intro)

- [Libraries](#libraries)

- [Datetime Dataframe](#datetime)

- [Pageviews](#pageviews)
    - [BW](#bw)
        - [Open BW dataset](#open_bw)
        - [Customize BW dataset](#customize_bw)
    - [Adwords](#adwords)
        - [Open Adwords dataset](#open_ads)
        - [Customize Ads dataset](#customize_ads)
    - [Merge BW and Adwords data](#merge_bw_ads)
    
- [References url](#url)
    - [Open References url dataset](#open_url)
    - [Merge References url and Top 50 references](#merge_url_top50)
    - [Customize References url dataset](#customize_url)

- [Looping through a dictionary](#looping)
    - [Looping through all key-value pairs](#loop_all_keys_values)
    - [Looping through all keys in a dictionary](#loop_all_keys)
    - [Looping through all values in a dictionary](#loop_all_values)
    - [Looping through a dictionary in order](#looping_in_order)
    - [Exercises](#exercises_looping)
    
- [Nesting](#nesting)
    - [Lists in a dictionary](#lists_in_dictionary)
    - [Dictionaries in a dictionary](#dictionaries_in_dictionary)
    - [An important note about nesting](#important_note)
    - [Exercises](#exercises_nesting)
    
- [Overall Challenges](#overall_challenges)

<a name='intro'></a>Introduction
===
EXPLICAR TODO Y PONER FOTO

<a name='libraries'></a>Libraries
===
First of all, it is necessary to import the required libraries to develop all the steps correctly of this notebbook.

In [2]:
import os
import io
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

#seguramente haya que pones mas librerias

#display maximo columnas

#IMPORTAR WARNINGS

<a name='datetime'></a>  Datetime Dataframe
===
An inconvenient of the data selected for this study is missing values. 
For example, the advertising dataset has a missing value during a specific period when the reference has no impressions a day (it is possible to check an example in the photography atach)

### FOTO!!!!


For this reason, I am going to create a dataframe of a period of time and save it as csv with the purpose of merging with miss values from other datasets that I will work during this study.

In [3]:
path = '/home/miguel/my_project_python/my_project_env/tfm/shopping/notebook'
os.chdir(path)

In [4]:
%run -i 'date_dataframe.py'

<a name='pageviews'></a>Pageviews
===

In this notebook, I am going to merge all datasets link to pageviews information in order to create for each reference a csv file with the following data:
    - Reference
    - Date
    - Pageviews

<a name='pageviews'></a>Pageviews
===

In this notebook, I am going to merge all datasets link to pageviews information in order to create for each reference a csv file with the following data:
    - Reference
    - Date
    - Pageviews

First of all, I set the directory for the rest of notebook

In [5]:
path = '/home/miguel/my_project_python/my_project_env/tfm/shopping'
os.chdir(path)

Then, I open the files required:

In [6]:
#select directory
path_reference_url = './data/customize/top_50'
path_datetime = './data/raw/datetime'


#load files
df_reference_url = pd.read_csv(path_reference_url+'/references_url.csv', sep='\t') 
df_datetime = pd.read_csv(path_datetime+'/datetime_df.csv')    #hacer funcion para datetime


#select columns
column_interest_reference = ['Reference', 'URL']
df_reference_url = df_reference_url[column_interest_reference]


column_interest_datetime = ['Dates']
df_datetime = df_datetime[column_interest_datetime]


#parse datetime
df_datetime['Dates'] = pd.to_datetime(df_datetime['Dates'], format = '%Y-%m-%d')

I check the data

In [7]:
df_reference_url.head(5)

,Reference,URL
0,122532,/espejo-pie-nordico-blanco-plastico-dormitorio...
1,51148,/consola-3-cajones-y-4-puertas-oriental-rojo-m...
2,51149,/consola-6-cajones-y-2-puertas-oriental-rojo-m...
3,51136,/consola-3-cajones-y-4-puertas-oriental-negro-...
4,51140,/mesilla-de-noche-1-cajon-y-2-puertas-oriental...


In [8]:
df_datetime.head(5)

,Dates
0,2018-09-03
1,2018-09-04
2,2018-09-05
3,2018-09-06
4,2018-09-07


After checking previous dataframe, I create a function to cover mainly the following steps:

- Open a csv file as dataframe
- Clean data from that file (rename columns, parse to datetime, etc.)
- Merge that dataframe with other dataframes
- Fill missing values
- Return the finaly dataframe

In [9]:
def pageview_missing_values(path,file):
    
    #os.chdir(path)
    
    df_pageview = pd.read_csv(path+file)
    
    df_pageview = df_pageview.rename(index=str, columns={'Day Index':'Dates', 'Page Views': 'Page_Views'})
    
    column_interest_pageview = ['Page', 'Dates', 'Page_Views']
    df_pageview = df_pageview[column_interest_pageview]   
    
    df_pageview['Dates'] = pd.to_datetime(df_pageview['Dates'], format = '%d/%m/%Y')
    
    merge_pageview = df_pageview.merge(df_datetime,
        how='right',
        left_on=['Dates'],
        right_on=['Dates'])
    
    merge_pageview['Page'].fillna(method='ffill', inplace = True)
    merge_pageview['Page_Views'].fillna(0, inplace = True)
    
    merge_url = merge_pageview.merge(df_reference_url,
        how='inner',
        left_on=['Page'],
        right_on=['URL'])
    
    column_interest_reference = ['Reference', 'Dates', 'Page_Views']
    merge_url = merge_url[column_interest_reference].sort_values(by='Dates')
        
    return merge_url

Now, I need to make a list to save the required data

In [10]:
lista = df_reference_url['Reference'].apply(str)+'_url.csv'
c = lista.values.tolist()

Finally, I make a for loop where I apply the funciont "pageview_missing_values" and save the dataframes as csv in a new directory

In [11]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

In [12]:
for i in range(len(c)):
    
    path_raw_data = './data/raw/pageview/'
    
    df_test = pageview_missing_values(path_raw_data, c[i])
    file = c[i]
    
    path_save_data = './data/customize/pageviews/'
    #os.chdir(path)
    
    df_test.to_csv(path_save_data + file)

<a name='bw'></a>BW
===

In [13]:
def open_bw(file,sheet_name, path):

    #open the file xlsx    
    df_bw = pd.read_excel(path+file, sheet_name)
    
    #rename the file columns correctly
    df_bw = df_bw.rename(index=str, columns={'Referencia':'Reference', 'T Día natural':'Date', 
                                             'Valor neto pedidos':'Net_Incomes', 
                                             'Cantidad en unidades (pedidos)':'Units_sold'})
    
    #parse date columns
    df_bw['Date'] = pd.to_datetime(df_bw['Date'], format = '%d.%m.%Y')
    
    return df_bw

In [14]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

In [15]:
#open the file
df_bw = open_bw('BW.xlsx', 'BW', './data/raw/bw/')

#head the file
df_bw.head(5)

,Reference,Name,Date,Net_Incomes,Units_sold
0,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-09-11,18.93,1
1,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-09-21,18.93,1
2,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-08,16.08,1
3,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-16,16.08,1
4,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-21,16.08,1


In [16]:
df_bw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37614 entries, 0 to 37613
Data columns (total 5 columns):
Reference      37614 non-null int64
Name           37614 non-null object
Date           37614 non-null datetime64[ns]
Net_Incomes    37614 non-null float64
Units_sold     37614 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.7+ MB


In [17]:
df_bw.shape

(37614, 5)

I also change "Reference" column from int64 to string in order to create a list later

In [18]:
df_bw['Reference'] = df_bw['Reference'].apply(str)

In [19]:
#create unique list of names
#UniqueNames = data.Names.unique()
referencess = df_bw['Reference'].unique().tolist()

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame for elem in referencess}

for key in DataFrameDict.keys():
    DataFrameDict[key] = df_bw[:][df_bw.Reference == key]

I use a reference id as example to check if it is working correctly

In [20]:
DataFrameDict['4623']

,Reference,Name,Date,Net_Incomes,Units_sold
0,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-09-11,18.93,1
1,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-09-21,18.93,1
2,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-08,16.08,1
3,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-16,16.08,1
4,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-21,16.08,1


As we can see, it works well!

In [21]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

In [22]:
#set directory
path = './data/customize/top_50/'

#open the file 'top_50'
df_top_50 = pd.read_csv(path+'references_url.csv', sep='\t')

#select columns
column_interest_reference = ['Reference']
df_top_50 = df_top_50[column_interest_reference]

In [23]:
df_top_50.head()

,Reference
0,122532
1,51148
2,51149
3,51136
4,51140


In [24]:
df_top_50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 1 columns):
Reference    50 non-null int64
dtypes: int64(1)
memory usage: 480.0 bytes


I create another list using last dataset (I apply string to "Reference" column as I have also made before)

In [25]:
lista = df_top_50['Reference'].apply(str)
c = lista.values.tolist()

I make a for loop to:

- Read each component of the list and check its data from DataFrameDict.
- Set the directory to storage the data
- Save a csv file for each reference

In [26]:
for i in range(len(c)):
    
    df_test = DataFrameDict[c[i]]
    
    file = c[i]
    
    path = './data/customize/bw/'
        
    df_test.to_csv(path + file + '.csv')

Check the saved data

In [27]:
df_101318 = pd.read_csv('./data/customize/bw/101318.csv', sep=',', error_bad_lines=True)
df_101318.head()

,Unnamed: 0,Reference,Name,Date,Net_Incomes,Units_sold
0,21074,101318,SILLA MARRÓN PU/METAL SALÓN,2017-10-05,151.53,3
1,21075,101318,SILLA MARRÓN PU/METAL SALÓN,2017-10-12,118.85,2
2,21076,101318,SILLA MARRÓN PU/METAL SALÓN,2017-10-15,202.05,4
3,21077,101318,SILLA MARRÓN PU/METAL SALÓN,2018-02-06,44.55,1
4,21078,101318,SILLA MARRÓN PU/METAL SALÓN,2018-02-22,237.69,4


I create function "bw_missing_values" to be able to achieve following goals:

- Open some files that I will work
- Make the required data wrangling in this dataframes
- Work specificly in each reference
- Return a dataframe with the main data in each case

In [28]:
def bw_missing_values(path1, path2, file):
    
    #set directory and open datetime_df file
    #os.chdir(path1)
    df_datetime = pd.read_csv(path1 + 'datetime_df.csv') ###ABRIR CON LA FUNCION
    
    #selecto column interest and parse data to datetime
    column_interest_datetime = ['Dates']
    df_datetime = df_datetime[column_interest_datetime]
    df_datetime['Dates'] = pd.to_datetime(df_datetime['Dates'], format = '%Y-%m-%d')
    
        
    #set directory to open the reference file
    #os.chdir(path2)
    
    #open the csv file for each reference
    df_bw_ref = pd.read_csv(path2 + c[i] + '.csv')
        
    #data wranggling of this data        
    df_bw_ref['Date'] = pd.to_datetime(df_bw_ref['Date'], format = '%Y-%m-%d')
    df_bw_ref = df_bw_ref.rename(index=str, columns={'Net Incomes': 'Net_Incomes', })
        
    #select columns 
    column_interest_bw = ['Reference', 'Name', 'Date', 'Net_Incomes', 'Units_sold']
    df_bw_ref = df_bw_ref[column_interest_bw]
        
    #merge with df_datetime
    merge_bw = df_bw_ref.merge(df_datetime,
        how='right',
        left_on=['Date'],
        right_on=['Dates'])
        
    #fill NaN values
    merge_bw['Reference'].fillna(method='ffill', inplace = True)
    merge_bw['Name'].fillna(method='ffill', inplace = True)
    merge_bw.fillna(0, inplace = True)
    
    #sort values by Dates
    merge_bw = merge_bw.sort_values(by='Dates')
    
    #select required columns
    column_interest_merge = ['Reference', 'Name', 'Dates', 'Net_Incomes', 'Units_sold']
    merge_bw = merge_bw[column_interest_merge]   

        
    return merge_bw

I apply the previous function using a for loop

In [29]:
path_datetime = './data/raw/datetime/'
path_original_data = './data/customize/bw/'

for i in range(len(c)):
    df_test = bw_missing_values(path_datetime, path_original_data, c[i])
    file = c[i]
    
    path = './data/customize/merge_bw_datetime/'
    #os.chdir(path)
    
    df_test.to_csv(path + file + '.csv')

In [30]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

In [31]:
df_101318 = pd.read_csv('./data/customize/merge_bw_datetime/101318.csv', sep=',', error_bad_lines=True)
df_101318.head()

,Unnamed: 0,Reference,Name,Dates,Net_Incomes,Units_sold
0,19,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-03,0.0,0.0
1,20,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-04,0.0,0.0
2,21,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-05,0.0,0.0
3,22,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-06,0.0,0.0
4,23,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-07,0.0,0.0


<a name='adwords'></a>Adwords
==

In this notebook, I am going to work with adwords data:
    - Load datasets.
    - Concatenate these datasets into one main dataset
    - Clean data (rename, select columns, etc.)
    - Select specific references and their data from previous dataset.
    - Split the main dataframe into multiple dataframes (a dataframe per reference)
    - Merge multiples dataframes with "datatime_df"
    - Save multiples dataframes correctly as csv files

I open adwords data. I have to open three files according to different time periods:
- TFM_Datos_2017_18_1.csv
- TFM_Datos_2017_18_2.csv
- TFM_Datos_2018_19.csv

In [32]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

-------
QUITAR PRIMER FICHERO
SOLOS LOS DOS ULTIMOS Y FILTRAR
-------

In [33]:
#path = './data/raw/adwords/'

extension = 'csv'
all_filenames = [i for i in glob.glob('./data/raw/adwords/*.{}'.format(extension))]

#combine all files in the list
df_adwords = pd.concat([pd.read_csv(f, sep=";", error_bad_lines=True) for f in all_filenames])

/home/miguel/my_project_python/my_project_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [34]:
df_adwords.head(5)

,ID de producto,Campaña,ID de la campaña,Tipo de producto (primer nivel),Tipo de producto (segundo nivel),Día,Etiqueta personalizada 1,CPC máximo predeterminado del grupo de anuncios,Moneda,Impresiones,Clics,CTR,CPC medio,Coste,Conversiones,Todas las conversiones,Valor de conv.,Valor de todas las conversiones,Valor conv./coste
0,123624,Shop_Cocina y comedor_N1_Y18_W37,1559515409,cocina y comedor,tazas de café y mugs,12/10/2018,10,"0,15",EUR,1,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
1,107553,Shop_Smart_Lámparas_Y18_W45,1623248621,lámparas e iluminación,lámparas de techo,31/01/2019,40,"0,01",EUR,2,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,87810,Shop_Navidad_Estrellas de navidad_Y18_W12,1323457572,decoración de navidad,estrellas de navidad,17/10/2018,20,"0,2",EUR,1,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,105114,Shop_Decoración_N1_Y18_W37,1559890739,decoración para tu casa,figuras decorativas,25/10/2018,30,"0,15",EUR,4,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,111017,Shop_Navidad_Portavelas navideños_Y18_W12,1323682318,decoración de navidad,portavelas navideños,21/10/2018,20,"0,2",EUR,8,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [35]:
df_adwords.shape

(1026615, 19)

In [36]:
df_adwords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1026615 entries, 0 to 579075
Data columns (total 19 columns):
ID de producto                                     1026615 non-null int64
Campaña                                            1026615 non-null object
ID de la campaña                                   1026615 non-null int64
Tipo de producto (primer nivel)                    1026615 non-null object
Tipo de producto (segundo nivel)                   1026615 non-null object
Día                                                1026615 non-null object
Etiqueta personalizada 1                           1026615 non-null object
CPC máximo predeterminado del grupo de anuncios    1026615 non-null object
Moneda                                             1026615 non-null object
Impresiones                                        1026615 non-null int64
Clics                                              1026615 non-null int64
CTR                                                1026615 non-null

Firstly, I rename the columns

In [37]:
df_adwords.columns

Index(['ID de producto', 'Campaña', 'ID de la campaña',
       'Tipo de producto (primer nivel)', 'Tipo de producto (segundo nivel)',
       'Día', 'Etiqueta personalizada 1',
       'CPC máximo predeterminado del grupo de anuncios', 'Moneda',
       'Impresiones', 'Clics', 'CTR', 'CPC medio', 'Coste', 'Conversiones',
       'Todas las conversiones', 'Valor de conv.',
       'Valor de todas las conversiones', 'Valor conv./coste'],
      dtype='object')

In [38]:
df_adwords = df_adwords.rename(index=str, 
            columns={'ID de producto':'Reference', 'Tipo de producto (primer nivel)': 'CatN1',
                     'Tipo de producto (segundo nivel)':'CatN2', 'Día':'Date', 'Etiqueta personalizada 1': 'Cat_Price', 
                     'CPC máximo predeterminado del grupo de anuncios': 'CPC_max', 'CPC medio': 'CPC_medio', 
                     'Impresiones':'Impressions', 'Coste':'Cost', 'Conversiones': 'Conversions',
                    'Todas las conversiones': 'All_Conversions', 'Valor de conv.': 'Ads_Income', 
                    'Valor de todas las conversiones': 'Ads_Income_All', 'Valor conv./coste': 'ROAS_Ads'})

Then, I select the columns of our interest for this study

In [39]:

column_interest_adwords = ['Reference', 'CatN1', 'CatN2', 'Date', 'Cat_Price', 'CPC_medio', 'Impressions', 
                           'Clics', 'Cost', 'Conversions', 'All_Conversions', 'Ads_Income', 'Ads_Income_All',
                          'ROAS_Ads']
df_adwords = df_adwords[column_interest_adwords]

#Change reference value from int to string in order to save references id as list later
df_adwords['Reference'] = df_adwords['Reference'].apply(str)

#Parse time to 'Date' column
df_adwords['Date'] = pd.to_datetime(df_adwords['Date'], format = '%d/%m/%Y')

#Change to float type
#float_columns = ['CPC_medio', 'Cost', 'Conversions', 'All_Conversions', 'Ads_Income', 'Ads_Income_All']
#df_adwords['CPC_max'] = df_adwords['CPC_max'].str.replace(',','.').astype(float)
df_adwords['CPC_medio'] = df_adwords['CPC_medio'].str.replace(',','.').astype(float)
df_adwords['Cost'] = df_adwords['Cost'].str.replace(',','.').astype(float)
df_adwords['Conversions'] = df_adwords['Conversions'].str.replace(',','.').astype(float)
df_adwords['All_Conversions'] = df_adwords['All_Conversions'].str.replace(',','.').astype(float)
df_adwords['Ads_Income'] = df_adwords['Ads_Income'].str.replace(',','.').astype(float)
df_adwords['Ads_Income_All'] = df_adwords['Ads_Income_All'].str.replace(',','.').astype(float)
df_adwords['ROAS_Ads'] = df_adwords['ROAS_Ads'].str.replace('.','').str.replace(',','.').astype(float)

In [40]:
df_adwords.head(5)

,Reference,CatN1,CatN2,Date,Cat_Price,CPC_medio,Impressions,Clics,Cost,Conversions,All_Conversions,Ads_Income,Ads_Income_All,ROAS_Ads
0,123624,cocina y comedor,tazas de café y mugs,2018-10-12,10,0.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0
1,107553,lámparas e iluminación,lámparas de techo,2019-01-31,40,0.0,2,0,0.0,0.0,0.0,0.0,0.0,0.0
2,87810,decoración de navidad,estrellas de navidad,2018-10-17,20,0.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0
3,105114,decoración para tu casa,figuras decorativas,2018-10-25,30,0.0,4,0,0.0,0.0,0.0,0.0,0.0,0.0
4,111017,decoración de navidad,portavelas navideños,2018-10-21,20,0.0,8,0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df_adwords.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1026615 entries, 0 to 579075
Data columns (total 14 columns):
Reference          1026615 non-null object
CatN1              1026615 non-null object
CatN2              1026615 non-null object
Date               1026615 non-null datetime64[ns]
Cat_Price          1026615 non-null object
CPC_medio          1026615 non-null float64
Impressions        1026615 non-null int64
Clics              1026615 non-null int64
Cost               1026615 non-null float64
Conversions        1026615 non-null float64
All_Conversions    1026615 non-null float64
Ads_Income         1026615 non-null float64
Ads_Income_All     1026615 non-null float64
ROAS_Ads           1026615 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(2), object(4)
memory usage: 117.5+ MB


In [42]:
df_adwords.shape

(1026615, 14)

<a name='adwords'></a>Merge data
==

In [43]:
#HACER DATAFRAME AL PRINCIPIO Y AQUI SOLO LISTA

In [44]:
top_50_list = df_reference_url[['Reference']]
top_50_list['Reference'] = top_50_list['Reference'].apply(str)


date_dataframe.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [45]:
top_50_list.head(5)

,Reference
0,122532
1,51148
2,51149
3,51136
4,51140


I merge both dataframes

In [46]:
merge_adwords = df_adwords.merge(top_50_list,
    how='right',
    left_on=['Reference'],
    right_on=['Reference'])

In [47]:
merge_adwords.shape

(14627, 14)

In [48]:
round(df_adwords.shape[0]/merge_adwords.shape[0],0)

70.0

The new dataframe is 85x times smaller in number of rows (both maintain the same colums)

After simplifying the dataset, I is less hard to the computer splitting dataframes into different dataframes

In [49]:
#Create unique list of names
references = merge_adwords['Reference'].unique().tolist()

#Create a dataframe dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame for elem in references}

#Create a loop for to read all rows and save according to the key
for key in DataFrameDict.keys():
    DataFrameDict[key] = merge_adwords[:][merge_adwords.Reference == key]

I use a reference id as example to check its functionality

In [50]:
DataFrameDict['101318']

,Reference,CatN1,CatN2,Date,Cat_Price,CPC_medio,Impressions,Clics,Cost,Conversions,All_Conversions,Ads_Income,Ads_Income_All,ROAS_Ads
7650,101318,muebles,sillas,2018-12-13,80,0.37,3313,17,6.36,0.00,0.00,0.00,0.00,0.00
7651,101318,muebles,sillas,2018-12-03,80,0.41,2677,12,4.91,0.00,0.00,0.00,0.00,0.00
7652,101318,muebles,sillas,2018-11-21,80,0.61,4851,40,24.43,0.69,2.19,25.27,173.07,1.03
7653,101318,muebles,sillas,2018-12-17,80,0.35,4946,14,4.96,0.00,0.00,0.00,0.00,0.00
7654,101318,muebles,sillas,2019-01-13,80,0.44,6457,36,15.95,0.00,0.00,0.00,0.00,0.00
7655,101318,muebles,sillas,2018-12-06,80,0.43,2072,19,8.21,0.00,0.00,0.00,0.00,0.00
7656,101318,muebles,sillas,2018-12-18,80,0.42,3340,22,9.22,0.00,0.00,0.00,0.00,0.00
7657,101318,muebles,sillas,2019-01-09,80,0.42,5945,40,16.89,1.00,2.00,64.70,129.40,3.83
7658,101318,muebles,sillas,2018-11-12,80,0.37,4845,37,13.61,0.00,0.00,0.00,0.00,0.00
7659,101318,muebles,sillas,2018-12-08,80,0.37,2362,29,10.59,0.40,0.80,53.77,107.61,5.08


Later, I load top the csv file "references_url.csv" to use its id values as filters to create the required dataframes per reference.

Then, I create another list using last dataset (I use "Reference" column as I have also made before)

In [56]:
lista = df_reference_url['Reference'].apply(str)
c = lista.values.tolist()

-------------------

# UTILIZAR FUNCION

-------------------

In [58]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

In [61]:
for i in range(len(c)):
    df_test = DataFrameDict[c[i]]
    file = c[i]
    
    path = './data/customize/adwords/'
    #os.chdir(path)
    
    df_test.to_csv(path + file + '.csv')

In [63]:
df_test_ads = pd.read_csv(path + '101318.csv', sep=',', error_bad_lines=True)

In [64]:
df_test_ads.head(5)

,Unnamed: 0,Reference,CatN1,CatN2,Date,Cat_Price,CPC_medio,Impressions,Clics,Cost,Conversions,All_Conversions,Ads_Income,Ads_Income_All,ROAS_Ads
0,7650,101318,muebles,sillas,2018-12-13,80,0.37,3313,17,6.36,0.00,0.00,0.00,0.00,0.00
1,7651,101318,muebles,sillas,2018-12-03,80,0.41,2677,12,4.91,0.00,0.00,0.00,0.00,0.00
2,7652,101318,muebles,sillas,2018-11-21,80,0.61,4851,40,24.43,0.69,2.19,25.27,173.07,1.03
3,7653,101318,muebles,sillas,2018-12-17,80,0.35,4946,14,4.96,0.00,0.00,0.00,0.00,0.00
4,7654,101318,muebles,sillas,2019-01-13,80,0.44,6457,36,15.95,0.00,0.00,0.00,0.00,0.00


In [65]:
df_test_ads.shape

(284, 15)

In [66]:
df_test_ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 15 columns):
Unnamed: 0         284 non-null int64
Reference          284 non-null int64
CatN1              284 non-null object
CatN2              284 non-null object
Date               284 non-null object
Cat_Price          284 non-null int64
CPC_medio          284 non-null float64
Impressions        284 non-null int64
Clics              284 non-null int64
Cost               284 non-null float64
Conversions        284 non-null float64
All_Conversions    284 non-null float64
Ads_Income         284 non-null float64
Ads_Income_All     284 non-null float64
ROAS_Ads           284 non-null float64
dtypes: float64(7), int64(5), object(3)
memory usage: 33.4+ KB


Later, I create function "ads_missing_values" to be able to achieve following goals:

Open some files that I will work
Make the required data wrangling in this dataframes
Work specificly in each reference
Return a dataframe with the main data in each case

In [67]:
def ads_missing_values(path1, path2, file):
    
    #set directory and open datetime_df file
    #os.chdir(path1)
    df_datetime = pd.read_csv(path1 + 'datetime_df.csv')  ### UTILIZAR FUNCION
    
    #selecto column interest and parse data to datetime
    column_interest_datetime = ['Dates']
    df_datetime = df_datetime[column_interest_datetime]
    df_datetime['Dates'] = pd.to_datetime(df_datetime['Dates'], format = '%Y-%m-%d')
    
        
    #set directory to open the reference file
    #os.chdir(path2)
    
    #open the csv file for each reference
    df_ads_ref = pd.read_csv(path2 + c[i] + '.csv') #quizas debo poner file en vez de c[i]
        
    #data wranggling of this data        
    df_ads_ref['Date'] = pd.to_datetime(df_ads_ref['Date'], format = '%Y-%m-%d')
    
        
    #merge with df_datetime
    merge_bw = df_ads_ref.merge(df_datetime,
        how='right',
        left_on=['Date'],
        right_on=['Dates'])
        
    #fill NaN values
    merge_bw['Reference'].fillna(method='ffill', inplace = True)
    merge_bw['CatN1'].fillna(method='ffill', inplace = True)
    merge_bw['CatN2'].fillna(method='ffill', inplace = True)
    merge_bw['Cat_Price'].fillna(method='ffill', inplace = True)
    merge_bw.fillna(0, inplace = True)
    
    #sort values by Dates
    merge_bw = merge_bw.sort_values(by='Dates')
    
    #select required columns
    merge_bw = merge_bw.drop(['Date', 'Unnamed: 0'], axis=1)
    
    #drop duplicates in 'Dates'
    merge_bw['Dates'] = merge_bw['Dates'].drop_duplicates(keep='first')

        
    return merge_bw


I apply the previous function using a for loop

In [68]:
path_datetime = './data/raw/datetime/'
path_transform_data = './data/customize/adwords/'

for i in range(len(c)):
    df_test = ads_missing_values(path_datetime, path_transform_data, c[i])
    file = c[i]
    
    path = './data/customize/merge_ads_datetime/'
    #os.chdir(path)
    
    df_test.to_csv(path + file + '.csv')

Finally, I check the data obtained

In [69]:
df_122335 = pd.read_csv(path + "122335.csv", sep=",", error_bad_lines=True)
df_122335.head(5)

,Unnamed: 0,Reference,CatN1,CatN2,Cat_Price,CPC_medio,Impressions,Clics,Cost,Conversions,All_Conversions,Ads_Income,Ads_Income_All,ROAS_Ads,Dates
0,199,122335.0,muebles,mesitas de noche,100.0,0.20,783.0,6.0,1.17,0.0,0.0,0.0,0.0,0.0,2018-09-03
1,213,122335.0,muebles,mesitas de noche,100.0,0.24,715.0,2.0,0.48,0.0,0.0,0.0,0.0,0.0,2018-09-04
2,198,122335.0,muebles,mesitas de noche,100.0,0.19,768.0,5.0,0.96,0.0,0.0,0.0,0.0,0.0,2018-09-05
3,193,122335.0,muebles,mesitas de noche,100.0,0.21,776.0,6.0,1.26,0.0,0.0,0.0,0.0,0.0,2018-09-06
4,207,122335.0,muebles,mesitas de noche,100.0,0.21,883.0,7.0,1.50,0.0,0.0,0.0,0.0,0.0,2018-09-07


In [70]:
df_122335.shape

(215, 15)

In [76]:
print(df_122335.shape)
print("----------------")
print(df_122335.info())

(215, 15)
----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 15 columns):
Unnamed: 0         215 non-null int64
Reference          215 non-null float64
CatN1              215 non-null object
CatN2              215 non-null object
Cat_Price          215 non-null float64
CPC_medio          215 non-null float64
Impressions        215 non-null float64
Clics              215 non-null float64
Cost               215 non-null float64
Conversions        215 non-null float64
All_Conversions    215 non-null float64
Ads_Income         215 non-null float64
Ads_Income_All     215 non-null float64
ROAS_Ads           215 non-null float64
Dates              210 non-null object
dtypes: float64(11), int64(1), object(3)
memory usage: 25.3+ KB
None


-----------------------

# FALTA INFO

-----------------------

<a name='bw'></a>Final Dataset
===

In this notebook, I am going to merge all data linked per each reference worked in previous notebook. Then, I am going to merge this datasets:
    - Adwords
    - BW
    - Pageviews

# PONER DIBUJO DE LO QUE VOY A HACER

Tengo 3 directorios

Por lo tanto, abro archivo top 50 y creo lista
Luego leo con un for loop la lista y hago:
    - Abrir para cada directorio el archivo correspondiente
    - Guardar dicho archivo como dataframe
    - merge de los 3 archivos para cada referencia
    - Data clean
    - Revisar
    - Guardar en carpeta final
    
    ¿Hacer funcion de eso?

In [77]:
def final_dataset(file, path_ads, path_bw, path_page):
    
    #set directory to open the adwords dataset
    #os.chdir(path_ads)
    df_ads = pd.read_csv(path_ads + file + '.csv', sep=',', error_bad_lines=True)
        
    
    #set directory to open the bw dataset
    #os.chdir(path_bw)
    df_bw = pd.read_csv(path_bw + file + '.csv', sep=',', error_bad_lines=True)
        
    
    #set directory to open the page dataset
    #os.chdir(path_page)
    df_page = pd.read_csv(path_page + file +'_url.csv', sep=',', error_bad_lines=True)
        
    
    #merge: i have different options        
    #from functools import reduce
    #dfs = [df0, df1, df2, dfN]
    #df_final = reduce(lambda left,right: pd.merge(left,right,on='name'), dfs)
        
    #pd.merge(pd.merge(df1,df2,on='name'),df3,on='name')
    df_final = df_ads.merge(df_bw,on='Dates').merge(df_page,on='Dates')
    
    return df_final
    

In [80]:
#path = '/home/miguel/my_project_python/my_project_env/tfm/datasets'
path_ads =  './data/customize/merge_ads_datetime/'
path_bw =   './data/customize/merge_ads_datetime/'
path_page = './data/customize/pageviews/'
path_save = './data/customize/merge_all/'
    

#for loop to apply for each reference
for i in range(len(c)):
    #df_test = DataFrameDict[c[i]]
    #df_test = rolling_data(columns, c[i], path1)
    df = final_dataset(c[i], path_ads, path_bw, path_page)
    file = c[i]
    
    #path = '/home/miguel/my_project_python/my_project_env/tfm/datasets/adwords/end_data'
    #os.chdir(path_file)
    
    df.to_csv(path_save + file + '.csv')

In [81]:
df_122335 = pd.read_csv(path_save + "122335.csv", sep=",", error_bad_lines=True)
df_122335.head(5)

,Unnamed: 0,Unnamed: 0_x,Reference_x,CatN1_x,CatN2_x,Cat_Price_x,CPC_medio_x,Impressions_x,Clics_x,Cost_x,...,Clics_y,Cost_y,Conversions_y,All_Conversions_y,Ads_Income_y,Ads_Income_All_y,ROAS_Ads_y,Unnamed: 0.1,Reference,Page_Views
0,0,199,122335.0,muebles,mesitas de noche,100.0,0.20,783.0,6.0,1.17,...,6.0,1.17,0.0,0.0,0.0,0.0,0.0,0,122335,9.0
1,1,213,122335.0,muebles,mesitas de noche,100.0,0.24,715.0,2.0,0.48,...,2.0,0.48,0.0,0.0,0.0,0.0,0.0,87,122335,0.0
2,2,198,122335.0,muebles,mesitas de noche,100.0,0.19,768.0,5.0,0.96,...,5.0,0.96,0.0,0.0,0.0,0.0,0.0,88,122335,0.0
3,3,193,122335.0,muebles,mesitas de noche,100.0,0.21,776.0,6.0,1.26,...,6.0,1.26,0.0,0.0,0.0,0.0,0.0,1,122335,19.0
4,4,207,122335.0,muebles,mesitas de noche,100.0,0.21,883.0,7.0,1.50,...,7.0,1.50,0.0,0.0,0.0,0.0,0.0,2,122335,9.0


# FALTA REVISAR QUE HE MEZCLADO PARA QUE SALGA MAL

# SIGUIENTE PASOS:
- 1)SELECCIONAR COLUMNAS QUE QUEREMOS
- 2)HACER ROLLING (Y JUSTIFICAR)
- 3)CREAR COLUMNAS
- 4)REVISAR TODO OK
- 5)CONCATENAR TODOS LOS FILES
- 6)FILTRAR POR ROAS, PAGEVIEWS E IMPRESION>0
- 7)COMPROBAR TODO OK
- 8)GUARDAR

In [ ]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)